# String distances




In [1]:
import nltk
import numpy as np
import pandas as pd
#import editdistance

In [2]:
words = nltk.corpus.words.words()

In [3]:
pd.DataFrame(words).to_csv("words_nltk.csv")

# Jaccard similarity


Jaccard Distance is a measure of dissimilarity for sets. The lower the dissimilarity, the more similar are two objects.

$$
\text{jaccard_similarity}(A,B) = \frac{| A \cap B |} {| A \cup B |} 
$$


$$
\text{jaccard_distance}(A,B) = 1 - \frac{| A \cap B |} {| A \cup B |} 
$$

In [4]:
def jaccard_similarity(s1,s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

def jaccard_distance(s1,s2):
    return 1- jaccard_similarity(s1,s2)

In [5]:
jaccard_similarity(set("exponential"),set("polynomial"))

0.5454545454545454

In [6]:
jaccard_similarity(set("exponential"),set("exponentia"))

0.8888888888888888

In [7]:
len(set("exponential").union(set("polynomial"))), len(set("exponential").intersection(set("polynomial")))

(11, 6)

In [8]:
set1 = set('mapping')
set2 = set('mappings')
jaccard_similarity(set1,set2), jaccard_distance(set1,set2), jaccard_distance(set1,set1)

(0.8571428571428571, 0.1428571428571429, 0.0)

In [9]:
set1 = set('mapping')
set2 = set('mappings')
nltk.jaccard_distance(set1, set2)

0.14285714285714285

What is the issue with the similarity? Order of the characters not matter

In [10]:
set1 = set('panmpi')
set2 = set('mapping')
jaccard_distance(set1,set2)

0.16666666666666663

In [11]:
set1 = set('mapping')
set2 = set('mappin')
nltk.jaccard_distance(set1, set2)

0.16666666666666666

In [12]:
words     =  nltk.corpus.words.words()
words_set = [set(w) for w in words]
len(words)

236736

In [13]:
query_word = set("guardin")
print("number of words",len(words))

distances = []
for word in words_set:
    ed = jaccard_distance(query_word, word)
    distances.append(ed)

number of words 236736


In [14]:
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is guardian


In [15]:
closest_words = [words[d] for d in np.argsort(distances)]
closest_words[0:10]

['unniggard',
 'gurniad',
 'guarding',
 'undaring',
 'guardian',
 'indiguria',
 'ungrained',
 'antidrug',
 'unarraigned',
 'underguardian']

Let us improve the returned closest words using the jaccard distance.

We can compute the jaccard distance using substrings of n characters from a string, if we do this we introduce local information about the order of substrings.

In [26]:
def build_n_chars(w, n_chars=2):
    return set([w[i:i+n_chars] for i in range(len(w)-n_chars)])

In [30]:
w = words[10]

In [32]:
build_n_chars(w,2)

{'Aa', 'ar', 'ni', 'on', 'ro'}

In [33]:
query_word_2chars = build_n_chars("guardin",2)
words_set_2chars = [build_n_chars(w,2) for w in words]

In [34]:
distances = []
for word in words_set_2chars:
    ed = jaccard_distance(query_word_2chars, word)
    distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is guardian


In [35]:
closest_words = [words[d] for d in np.argsort(distances)]
closest_words[0:10]

['guarding',
 'guardian',
 'guardo',
 'guarded',
 'guarder',
 'coguardian',
 'guardianly',
 'guardiancy',
 'guardingly',
 'guara']

# Edit distance

Given two strings `x` and `y` the edit distance is the cheapest possible sequence of **character edits** to transform `x` to `y`.

Character edits (or character transformations) are:

- Insert a character `c`
- Delete `c`
- Replace `c` by `c'`


In [36]:
mistake = "drauing" 
words = ['cat', 'draw', 'drawing', 'drought', 'linking',
        'living', 'dragon', 'handemore', 'eliot', 'queen']

distances = []
for word in words:
    ed = nltk.edit_distance(mistake, word)
    print("d({:10},{}) = {}".format(word,mistake,ed))
    distances.append(ed)

print("\nthe closest word is", words[np.argmin(distances)])

d(cat       ,drauing) = 6
d(draw      ,drauing) = 4
d(drawing   ,drauing) = 1
d(drought   ,drauing) = 4
d(linking   ,drauing) = 4
d(living    ,drauing) = 4
d(dragon    ,drauing) = 3
d(handemore ,drauing) = 9
d(eliot     ,drauing) = 6
d(queen     ,drauing) = 6

the closest word is drawing


In [37]:
def edit_distance_recursive(x,y):
    if len(x) ==0:
        return len(y)
    if len(y) == 0:
        return len(x)
    
    delta = 0 if x[-1] == y[-1] else 1
    return min(edit_distance_recursive(x[:-1],y[:-1]) + delta,
               edit_distance_recursive(x[:-1],y)      + 1,
               edit_distance_recursive(x,y[:-1])      + 1)
     

In [38]:
edit_distance_recursive("ab", "ac")

1

In [39]:
%%time
edit_distance_recursive("house", "hause")

CPU times: user 766 µs, sys: 0 ns, total: 766 µs
Wall time: 768 µs


1

In [40]:
%%time
dist_recursive = edit_distance_recursive("superman", "supermaniac")
print(f"distance by edit_distance_recursive {dist_recursive}\n")

distance by edit_distance_recursive 3

CPU times: user 1.13 s, sys: 9.25 ms, total: 1.14 s
Wall time: 1.14 s


In [41]:
t =  len(words)/(1.13*60*60)
print(f"time needed to compute the distance between a w and all vocab {t} hours")

time needed to compute the distance between a w and all vocab 0.0024582104228121925 hours


How come nltk needs less than 100 µs?

In [42]:
%%time
dist_nltk = nltk.edit_distance("superman", "supermaniac")
print(f"distance by nltk {dist_nltk}\n")

distance by nltk 3

CPU times: user 174 µs, sys: 42 µs, total: 216 µs
Wall time: 196 µs


Notice that the `edit_dinstance_recursive` calls itself with the same input values many times, making it very slow.

We can know the number of times it call itself using a global variable counter `n`. 

In [43]:
n = 0
def edit_distance_recursive(x,y):
    global n
    if len(x) ==0:
        return len(y)
    if len(y) == 0:
        return len(x)
    
    if x =="super" and y=="sup":
        n += 1
    
    delta = 0 if x[-1] == y[-1] else 1
    return min(edit_distance_recursive(x[:-1],y[:-1]) + delta,
               edit_distance_recursive(x[:-1],y)      + 1,
               edit_distance_recursive(x,y[:-1])      + 1)
     

In [44]:
edit_distance_recursive("superman", "supermaniac")
n

833

###### Other types of edit distance


You can put a diferent cost to the different operations performed when computing an edit distance

In [45]:
w1 = 'mapping'
w2 = 'mappings'
nltk.edit_distance(w1, w2,substitution_cost=1)

1

In [46]:
w1 = 'mapping'
w2 = 'mappink'
nltk.edit_distance(w1, w2,substitution_cost=2)

2

## Using a big list of words


Fast implementations are important when computing edit distances between lots of candidates

In [47]:
import editdistance

If we have a lot of words it's important to make sure we use a fast distance immplementation.

- `editdistance` provides a much faster implementation than `nltk`


In [48]:
words = nltk.corpus.words.words()
len(words)

236736

In [49]:
#%%time
mistake = "drauing" 
distances = []
for word in words:
    ed = nltk.edit_distance(mistake, word)
    distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is drawing


In [50]:
import editdistance

In [51]:
#%%time
mistake = "drauing" 
distances2 = []
for word in words:
    ed = editdistance.eval(mistake, word)
    distances2.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances2)])


the closest word is drawing


In [52]:
# both implementations provide the same results
distances2 == distances

True

## Implementing an edit distance using dynamic programming


The basic idea to implement the edit distance in an efficient way relies on understanding that if we have the edit distances between two prefixes of two strings we can use it to compute the edit distance between the prefixes appending them with a character.



###### Example

Compute `d("hi","ho")`

```
    ""    h      i
""  0     1      2 
h   1     d,i,s  d,i,s
o   2     d,i,s  d,i,s
```

Which ends up with 

```
    ""    h    i
""  0     1    2 
h   1     0    1 
o   2     1    1
```

Compute `d("hi","hill")`


```
    ""    h        i
""  0     1        2 
h   1     d,i,s    d,i,s
i   2     d,i,s    d,i,s
l   3     d,i,s    d,i,s
l   4     d,i,s    d,i,s
```

```
    ""    h        i
""  0     1        2 
h   1     1,1,0    d,i,s
i   2     d,i,s    d,i,s
l   3     d,i,s    d,i,s
l   4     d,i,s    d,i,s
```







```
    ""    h    i
""  0     1    2 
h   1     0    1 
i   2     1    0
l   3     2    1
l   4     3    2
```



##### Dynamic programming

1) Define subproblems

- $x[i:]$ and $y[j:]$ for all i and j
- #subproblems = $O(|x| |y|)$

2) Guess part of the overall solution

we want to map the first character of x to the first character of y.


```
x = [x_1 x_2 ...]
y = [y_1 y_2 ...]
```



3) Define a recurrence

4) Recurrence + memoization

5) Solve original problem

## Edit distance full algorithm

Initialization
```
D(i,0) = i
D(0,j) = j
```

That is:

```
    ""    ------------- x --------------- 
""  0      1      2               lx-1 lx
|   1      0      0               0    0
|   2      0      0               0    0
    .      .      .               .    .
y   .      .      .               .    .
    .      .      .               .    .
|   ly-1   0      0               0    0
|   ly     0      0               0    0
```



Recurrence
```
For each i in 1...M
    For each j in 1...N
        del_char = D(i-1,j) + 1
        ins_char = D(i,j-1) + 1
     
        if X[i] != Y[j]:
            Z = 2
        if X[i] = Y[j]
            Z = 0
         
        sub_char = D(i-1,j-1) + Z
        D(i,j) = min(del_char, ins_char, sub_char)
```

Termination

The result will be the last position of the array `D` once it is filled.

```
return D(N,M) 
```



In [53]:
def initialize_table(s1,s2):
    len_x = len(s1)
    len_y = len(s2)
    D = np.zeros((len_x+1,len_y+1))

In [54]:
s1 = "hill"
s2 = "hello"
len_x = len(s1)
len_y = len(s2)
D = np.zeros((len_x+1,len_y+1))
D[:,0] = range(len_x+1)
D[0,:] = range(len_y+1)
D

array([[0., 1., 2., 3., 4., 5.],
       [1., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0.]])

In [55]:
def memoization_table(s1,s2,D):
    colnames = ["empty"] + [x for x in s2]
    rownames = ["empty"] + [x for x in s1]
    print(pd.DataFrame(D, rownames, columns=colnames))

In [56]:
memoization_table(s1,s2,D)

       empty    h    e    l    l    o
empty    0.0  1.0  2.0  3.0  4.0  5.0
h        1.0  0.0  0.0  0.0  0.0  0.0
i        2.0  0.0  0.0  0.0  0.0  0.0
l        3.0  0.0  0.0  0.0  0.0  0.0
l        4.0  0.0  0.0  0.0  0.0  0.0


In [57]:
D

array([[0., 1., 2., 3., 4., 5.],
       [1., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0.]])

In [58]:
s1 = "EXPONENTIAL"
s2 = "POLYNOMIAL"
len_x = len(s1)
len_y = len(s2)
D = np.zeros((len_x+1,len_y+1))
D[:,0] = range(len_x+1)
D[0,:] = range(len_y+1)
D

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [59]:
s1 = "EXPONENTIAL"
s2 = "POLYNOMIAL"
len_x = len(s1)
len_y = len(s2)
D = np.zeros((len_x+1,len_y+1))
D[:,0] = range(len_x+1)
D[0,:] = range(len_y+1)
D

X = s1
Y = s2

w_sub = 1
w_del = 1
w_ins = 1

for i in range(1,len(X)+1):
    for j in range(1,len(Y)+1):
        
        del_char = D[i-1,j] + w_del
        ins_char = D[i,j-1] + w_ins
        
        if X[i-1] == Y[j-1]:
            Z = 0
        else:
            Z = w_sub
        sub_char = D[i-1,j-1] + Z

        D[i,j] = min(del_char, ins_char, sub_char)



In [60]:
memoization_table(X,Y,D)

       empty     P    O    L    Y    N    O    M    I    A     L
empty    0.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
E        1.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
X        2.0   2.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
P        3.0   2.0  3.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
O        4.0   3.0  2.0  3.0  4.0  5.0  5.0  6.0  7.0  8.0   9.0
N        5.0   4.0  3.0  3.0  4.0  4.0  5.0  6.0  7.0  8.0   9.0
E        6.0   5.0  4.0  4.0  4.0  5.0  5.0  6.0  7.0  8.0   9.0
N        7.0   6.0  5.0  5.0  5.0  4.0  5.0  6.0  7.0  8.0   9.0
T        8.0   7.0  6.0  6.0  6.0  5.0  5.0  6.0  7.0  8.0   9.0
I        9.0   8.0  7.0  7.0  7.0  6.0  6.0  6.0  6.0  7.0   8.0
A       10.0   9.0  8.0  8.0  8.0  7.0  7.0  7.0  7.0  6.0   7.0
L       11.0  10.0  9.0  8.0  9.0  8.0  8.0  8.0  8.0  7.0   6.0


In [61]:
# edit distance
D[-1,-1]

6.0

## Assuming same cost for all edits

In [62]:
def create_memoization_table(X,Y):

    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x+1,len_y+1))
    
    for i in range(len(X)+1):
        for j in range(len(Y)+1):

            if i == 0:
                D[i][j] = j    

            elif j == 0:
                D[i][j] = i  

            elif X[i-1] == Y[j-1]: 
                D[i][j] = D[i-1][j-1]

            else:
                D[i][j] = 1+min(D[i][j-1],      # Insert 
                                D[i-1][j],      # Remove 
                                D[i-1][j-1])    # Replace 
    return D

In [63]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"
D = create_memoization_table(x,y)
memoization_table(x,y,D)
D[-1,-1]

       empty     P    O    L    Y    N    O    M    I    A     L
empty    0.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
E        1.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
X        2.0   2.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
P        3.0   2.0  3.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0
O        4.0   3.0  2.0  3.0  4.0  5.0  5.0  6.0  7.0  8.0   9.0
N        5.0   4.0  3.0  3.0  4.0  4.0  5.0  6.0  7.0  8.0   9.0
E        6.0   5.0  4.0  4.0  4.0  5.0  5.0  6.0  7.0  8.0   9.0
N        7.0   6.0  5.0  5.0  5.0  4.0  5.0  6.0  7.0  8.0   9.0
T        8.0   7.0  6.0  6.0  6.0  5.0  5.0  6.0  7.0  8.0   9.0
I        9.0   8.0  7.0  7.0  7.0  6.0  6.0  6.0  6.0  7.0   8.0
A       10.0   9.0  8.0  8.0  8.0  7.0  7.0  7.0  7.0  6.0   7.0
L       11.0  10.0  9.0  8.0  9.0  8.0  8.0  8.0  8.0  7.0   6.0


6.0

In [64]:
x = "Elliot"
y = "Elia"
D = create_memoization_table(x,y)
memoization_table(x,y,D)
D[-1,-1]

       empty    E    l    i    a
empty    0.0  1.0  2.0  3.0  4.0
E        1.0  0.0  1.0  2.0  3.0
l        2.0  1.0  0.0  1.0  2.0
l        3.0  2.0  1.0  1.0  2.0
i        4.0  3.0  2.0  1.0  2.0
o        5.0  4.0  3.0  2.0  2.0
t        6.0  5.0  4.0  3.0  3.0


3.0

In [65]:
x = "hi"
y = "hill"
D = create_memoization_table(x,y)
memoization_table(x,y,D)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))

       empty    h    i    l    l
empty    0.0  1.0  2.0  3.0  4.0
h        1.0  0.0  1.0  2.0  3.0
i        2.0  1.0  0.0  1.0  2.0

The distance between hi and hill is 2.0


##### Timing implementation

In [66]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"


In [67]:
def edit_distance_fast(x,y):
    D = create_memoization_table(x,y)
    return D[-1,-1]

In [68]:
%%timeit
edit_distance_fast(x,y)

164 µs ± 945 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [69]:
%%timeit
nltk.edit_distance(x,y)

72.8 µs ± 762 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [70]:
nltk.edit_distance(x,y) == edit_distance_fast(x,y)

True

### Different costs per operation

In [71]:
def memoization_table_weighted(X,Y):

    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x+1,len_y+1))
    D[:,0] = range(len_x+1)
    D[0,:] = range(len_y+1)

    w_sub = 1
    w_del = 1
    w_ins = 1

    for i in range(1,len_x+1):
        for j in range(1,len_y+1):
            del_char = D[i-1,j] + w_del
            ins_char = D[i,j-1] + w_ins

            if X[i-1] == Y[j-1]:
                Z = 0
            else:
                Z = w_sub
            sub_char = D[i-1,j-1] + Z

            D[i,j] = min(del_char, ins_char, sub_char)

    return D

In [72]:
x = "Elliot"
y = "Elia"
D = memoization_table_weighted(x,y)
memoization_table(x,y,D)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))

       empty    E    l    i    a
empty    0.0  1.0  2.0  3.0  4.0
E        1.0  0.0  1.0  2.0  3.0
l        2.0  1.0  0.0  1.0  2.0
l        3.0  2.0  1.0  1.0  2.0
i        4.0  3.0  2.0  1.0  2.0
o        5.0  4.0  3.0  2.0  2.0
t        6.0  5.0  4.0  3.0  3.0

The distance between Elliot and Elia is 3.0


### Timing 

In [73]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"

In [74]:
%%timeit
D = memoization_table_weighted(x,y)

186 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [75]:
%%timeit
nltk.edit_distance(x,y)

72 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Backtracking edits performed


Let us start with an example

Start from the right bottom position

```
       empty     h     i     l      l
empty    0.0   1.0   2.0    3.0   4.0
h        1.0   0.0   1.0    2.0   3.0
i        2.0   1.0   0.0    1.0  [2.0]
```

Find the nearby adjacent position with the lowest value

```
       empty     h     i     l      l
empty    0.0   1.0   2.0    3.0   4.0
h        1.0   0.0   1.0    2.0   3.0
i        2.0   1.0   0.0  [1.0]  [2.0]
```


From that position find the nearby adjacent position with the lowest value
```
       empty     h     i     l      l
empty    0.0   1.0   2.0    3.0   4.0
h        1.0   0.0   1.0    2.0   3.0
i        2.0   1.0  [0.0]  [1.0]  [2.0]
```


From that position find the nearby adjacent position with the lowest value
```
       empty     h     i     l      l
empty    0.0   1.0   2.0    3.0   4.0
h        1.0 [0.0]   1.0    2.0   3.0
i        2.0   1.0  [0.0]  [1.0]  [2.0]
```

From that position find the nearby adjacent position with the lowest value
```
       empty        h          i        l        l
empty  [0.0]<-,   1.0        2.0      3.0      4.0
h        1.0   `-[0.0]<-,    1.0      2.0      3.0
i        2.0       1.0   `-[0.0]<---[1.0]<---[2.0]
```

You have reached position $(0,0)$.

The edit path is 

$$ 
\mathrm{editpath} = (2,4),(2,3),(2,2),(1,1),(0,0)
$$

From the edit path and the edit distance matrix we can retrieve the generated edits to transform $x=hi$ to $y=hill$.  To do so we traverse the path and check if consecutive positions in the edit matrix have different values. 

- If values are the same it means there was no edit.


- If values are different it means we did and edit, we can record the edit.
    - If `origin[0] == destination[0]` means that is a horizontal move (char addition)
    - If `origin[1] == destination[1]` means that is a vertical move   (char addition)
    

Let us do it for each step along the way. Let us denote by "origin" the position from which we start every step along the path. Let us denote by "destination" the position at which we go. For example, the first step along the path is $(2,4)\rightarrow(2,3)$ where the origin=$(2,4)$ and destination=$(2,3)$.

X = "hi"
Y = "hill"

- $(2,4)\rightarrow(2,3)$: 
    - Notice `M[2,4] = 2`, `M[2,3] = 1`. Then there was an edit.
    - If `origin[0] == destination[0]` means that is a horizontal move (char addition)
    - We added the char Y[3] = L
    
- $(2,3)\rightarrow(2,2)$: 
    - Notice `M[2,3] = 1`, `M[2,2] = 0`. Then there was an edit.
    - If `origin[0] == destination[0]` means that is a horizontal move (char addition)
    - We added the char Y[2] = L


- $(2,2)\rightarrow(1,1)$: 
    - Notice `M[2,2] = 0`, `M[1,1] = 0`. No edit


- $(1,1)\rightarrow(0,0)$: 
    - Notice `M[1,1] = 0`, `M[0,0] = 0`. No edit
    - (0,0) flags a stopping criteria
    


This answer from stack overflow has a detailed example

- https://stackoverflow.com/questions/5849139/levenshtein-distance-inferring-the-edit-operations-from-the-matrix



In [76]:
x = "hi"
y = "hill"
D = create_memoization_table(x,y)
memoization_table(x,y,D)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))

       empty    h    i    l    l
empty    0.0  1.0  2.0  3.0  4.0
h        1.0  0.0  1.0  2.0  3.0
i        2.0  1.0  0.0  1.0  2.0

The distance between hi and hill is 2.0


# Speeding up code

Simple example with cython

In [77]:
%load_ext cython

In [78]:
def fib(n):
    a = 0.
    b = 1.
    for i in range(n):
        a, b = a + b, a
    return a

In [79]:
%%cython --a
cpdef cy_fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [80]:
fib(10)

55.0

In [81]:
cy_fib(10)

55.0

In [82]:
import timeit

In [83]:
n_times = 100000
t_fib = timeit.timeit("fib(10)", setup="from __main__ import fib",number=n_times)
t_cyfib = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_fib_unit = t_fib/n_times

t_cyfib      = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_cyfib_unit = t_cyfib/n_times

print(" Python version took: {} sec\n Cython version took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib, t_cyfib, t_fib/t_cyfib))

print("\n Python version 1 run took: {} sec\n Cython version took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib_unit, t_cyfib_unit, t_fib_unit/t_cyfib_unit))

 Python version took: 0.05685630999960267 sec
 Cython version took: 0.0031541899998046574 sec
 Cython is 18x faster

 Python version 1 run took: 5.685630999960267e-07 sec
 Cython version took: 3.1541899998046577e-08 sec
 Cython is 18x faster


### Speeding up edit distance



##### Exercise fill in cy_create_memoization_table so that it returns the matrix filled to compute the edit distance

In [87]:
%%cython --a

import numpy as np
cimport cython

cpdef cy_create_memoization_table(str X, str Y):

    
    # TODO BEGIN ---------------------------
    
    
    
    # TODO END ---------------------------
    
    #return D

In [83]:
D1 = create_memoization_table(x,y)

In [84]:
D2 = cy_create_memoization_table(x,y)
D2 = np.asarray(D2)

In [85]:
t_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; create_memoization_table(x,y)",
                                           setup="import numpy as np; from __main__ import create_memoization_table",
                                          number=5000)

In [86]:
t_cy_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; cy_create_memoization_table(x,y)",
                                              setup="from __main__ import cy_create_memoization_table",
                                              number=5000)

In [87]:
t_nltk = timeit.timeit("x='exponential'; y='polynomial'; nltk.edit_distance(x,y)",
                        setup="import nltk ",
                        number=5000)

In [88]:
print(""" 
      Python version took: {} sec
      Cython version took: {} sec
      nltk   version took: {} sec
      Cython is {:.0f}x faster than python
      Cython is {:.0f}x faster than nltk
      """\
      .format(t_create_memoization_table, 
              t_cy_create_memoization_table,
              t_nltk, 
              t_create_memoization_table/t_cy_create_memoization_table,
              t_nltk/t_cy_create_memoization_table))

 
      Python version took: 0.8201289099997666 sec
      Cython version took: 0.010222451001027366 sec
      nltk   version took: 0.34801899699959904 sec
      Cython is 80x faster than python
      Cython is 34x faster than nltk
      


### Return to the experiment where we computed closest word


##### Exercise: Return the last component of the DynamicProgramming matrix containing the edit distance

In [91]:
%%cython --a

import numpy as np
cimport cython

cpdef edit_distance(str X, str Y):

    ### TODO BEGIN ----------------------------------
    
    
    ### TODO END ------------------------------------
    return dist


In [90]:
edit_distance("lik", "cat")

3

In [91]:
words = nltk.corpus.words.words()
len(words)

236736

In [92]:
%%time
mistake = "drauing" 
distances = []
for word in words:
    ed = nltk.edit_distance(mistake, word)
    distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is drawing
CPU times: user 10.3 s, sys: 28.2 ms, total: 10.3 s
Wall time: 10.3 s


In [109]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = editdistance.eval(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: user 451 ms, sys: 1.81 ms, total: 453 ms
Wall time: 452 ms


In [114]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = edit_distance(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: user 430 ms, sys: 1.56 ms, total: 432 ms
Wall time: 431 ms


In [116]:
distances == cy_distances

True

In [112]:
editdistance.eval("hi", "hi"), edit_distance("hi","hi")

(0, 0)

In [113]:
editdistance.eval("hi", "ho"), edit_distance("hi","ho")

(1, 1)

# BK tree

## How to create a BK-tree

- take any word from your set and use it in as your root node. Add word to the tree based on their distance to the root.


### Example

- Let us consider we started a tree with the word set [**book**, **books**, **cake**] then my tree would look like this if I started by making the word **book**:

```
       1 
book ----- books
    |
    | 4
    ----cake
```

- Add **boo**. Now we want to add the word **boo** which is at distace 1 from book. Notice though that there is already **book** at distance 1. The BK tree has to respect that every node have all children with different distances. If there is a colission (like we have now) then the new word must become a children of the node with which there was the colition. In this case, a children of **book**. The new weight from **books** to **boo** has to be `edit_dis(books,boo)=2`.

```
       1           2
book ----- books ----- boo
    |
    | 4
    ----cake
```

#### Adding [**cape**,**cart**,**boon**,**cook**].

- Add **cape**. Notice `edit_dis(books,cape)=4` but **cake** is altready at distance 4. Therefore **cape** needs to be a child of **cake** with a weighted edge of weight `edit_dis(cake,cape)=1`.
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
```

- Add **cart**. Notice `edit_dis(books,cart)=4`. Therefore **cart** needs to be a children of **cake** (or any subchildren of cake). Since `edit_dis(cake,cart)=2` then :
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **boon**. Notice  `edit_dis(book,boon)=1`. Therefore **boon** has to be a descendant of **books** (because books is a son of book with weight 1). Notice `edit_dis(books,boon)=2` but there is already node **boo** at distance 2. Therefore **boon** needs to be a descendant of **boo**. Notice `edit_dis(boo,boon)=1`.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                   
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **cook**.  Notice  `edit_dis(book,cook)=1` but book contains a descendant at distance 1, books, so cook has to be a dascendant of books.  Notice  `edit_dis(books,cook)=2` but books has a descendant at distance 2, **boo**, so cook has to be a descendant of boo.   Notice  `edit_dis(boo,cook)=2` and boo has no other descendant at distance 2. Therefore we can add cook as descendant of boo with weight 2.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                    |
    |                    |  2
    |                    ------ cook
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```


## How to code a BK tree

Now we would like to know how do we construct a bk-tree in python and store it. A natural way to store a tree is using many tuples concatenated. Since a tree 




##### Exercise: fill in the BKTree class to create a tree

In [192]:
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        
        # TODO BEGIN ------------------------------------------
        
        
        # TODO END --------------------------------------------
        

In [193]:
%%time
t = BKTree(edit_distance,words)

CPU times: user 6.74 s, sys: 8 ms, total: 6.75 s
Wall time: 6.76 s


In [194]:
len(t.tree)

2

In [200]:
branches_from_root = t.tree[1]
branch_23          = branches_from_root[23]
branch_24          = branches_from_root[24]

In [205]:
#branches_from_root[2]

In [199]:
branch_23

('anthropomorphologically',
 {20: ('blepharosphincterectomy', {16: ('epididymodeferentectomy', {})}),
  21: ('formaldehydesulphoxylic',
   {19: ('pericardiomediastinitis', {}),
    22: ('Pseudolamellibranchiata', {2: ('pseudolamellibranchiate', {})}),
    20: ('transubstantiationalist', {})}),
  19: ('gastroenteroanastomosis',
   {15: ('macracanthrorhynchiasis', {}),
    17: ('pancreaticoduodenostomy', {}),
    20: ('phenolsulphonephthalein', {})}),
  18: ('hematospectrophotometer',
   {22: ('scientificogeographical', {}), 19: ('thymolsulphonephthalein', {})}),
  13: ('pathologicohistological', {}),
  14: ('philosophicotheological', {})})

In [191]:
branch_24

('formaldehydesulphoxylate',
 {21: ('pathologicopsychological', {}),
  22: ('scientificophilosophical', {}),
  18: ('tetraiodophenolphthalein', {}),
  20: ('thyroparathyroidectomize', {})})

## Searching in a BK tree

Now in order to search all words that appear at distance less or equal than a tolerance T form a query word `q` we need to visit all nodes n that are at distance D(n,q)+-N.

Let us consider word q=caqe, T=1, candidates = [], search=[book]

Select candidate **book** from search=[book]
- edit_dist(book,caqe) = 4 => candidates is not updated
    - Crawl all children of book at distance I=[4-1,4+1]=[3,5]
    - There is a single node, search=[book,cake], inside I.
    - search = [book,cake]\book = [cake]
    
Select candidate **cake** from search=[cake]
- edit_dist(cake,caqe) = 1 => candidates =[cake]
    - Crawl all children of book at distance I=[1-1,1+1]=[0,2]
    - There are 2 possible nodes, search=[cape, cart]


Select candidate **cape** from search=[cape,cart]
- edit_dist(cape,caqe) = 1 => candidates =[cake, cape]
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Cape has no children
    - search = [cape, cart]\cape = [cart]
    
    
Select candidate **cart** from search=[cart]
- edit_dist(cart,caqe) = 2 => candidates is not updated
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Caqe has no children
    - search = [cart]\cart = []


Select candidate... There is no candidate! stop process. 

The resulting set of possible candidates at distance 1 are: [cape,cake].

Notice that we ended up computing 4 edit distances yet we have 8 nodes.



### Making queries 

In [355]:
visited_nodes = []
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        # TODO BEGIN ------------------------------------------
        
        
        # TODO END --------------------------------------------
        
    def _search_descendants(self, parent, n, distance, query_word):
        
        node_word, children_dict = parent
        dist_to_node = distance(query_word, node_word)
        visited_nodes.append(node_word)
        results = []
        if dist_to_node <= n:
            results.append((dist_to_node, node_word))
        
        for i in range(dist_to_node-n, dist_to_node+n+1):
            child = children_dict.get(i) # children_dict[i] can return keyerror
            if child is not None:
                results.extend(self._search_descendants(child, n, distance, query_word))
                
        return results
            
    def query(self, query_word, n):
        # sort by distance
        return sorted(self._search_descendants(self.tree, n, self.distfn, query_word))

In [356]:
%%time
t = BKTree(edit_distance,words)

CPU times: user 7.67 s, sys: 20 ms, total: 7.69 s
Wall time: 7.69 s


In [368]:
%%time
visited_nodes = []
print(t.query("senzorial", 1))

[(1, 'sensorial')]
CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 35.9 ms


In [370]:
len(visited_nodes), len(words)

(5241, 236736)

Compare it with the nltk.edit_distance

In [224]:
%%time
t_nltk = BKTree(nltk.distance.edit_distance, words)

CPU times: user 2min 34s, sys: 64 ms, total: 2min 34s
Wall time: 2min 35s


In [228]:
%%time
t_nltk.query("senzorial", 1)

CPU times: user 392 ms, sys: 0 ns, total: 392 ms
Wall time: 398 ms


[(1, 'sensorial')]


### Correcting a pharse

Notice that there are several words at the same distance. Picking the correct word is not the aim for today's class. The aim is to find a small list that contains the correct word.

In [379]:
print(t.query("wentt", 1))

[(1, 'went')]


Notice that we have an issue with "hamer" since it is (in edit distance) as close as "hammer"  than "haler".

We would like to, somehow, choose wisely among a set of candidates.

In [378]:
print(t.query("hamer", 1))

[(1, 'haler'), (1, 'hame'), (1, 'hamel'), (1, 'hammer'), (1, 'hammer'), (1, 'hamper'), (1, 'harmer'), (1, 'hater'), (1, 'haver'), (1, 'hawer'), (1, 'hazer'), (1, 'homer'), (1, 'namer'), (1, 'shamer'), (1, 'tamer')]


In [371]:
phrase = "the man wentt to the store to buy a hamer and some nals"

In [372]:
%%time
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = t_nltk.query(w,2)
        #import pdb;pdb.set_trace()
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)

print("Original:   ",phrase)
print("Corrected:  "," ".join(correction))
print("\nTimming")

Original:    the man wentt to the store to buy a hamer and some nals
Corrected:   the man went to the store to buy a haler and some hals

Timming
CPU times: user 1.98 s, sys: 0 ns, total: 1.98 s
Wall time: 1.98 s


In [353]:
%%time
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = t.query(w,2)
        #import pdb;pdb.set_trace()
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)


print("Original:   ",phrase)
print("Corrected:  "," ".join(correction))
print("\nTimming")

Original:    the man went to the store to buy a hamer and some nals
Corrected:   the man went to the store to buy a haler and some hals

Timming
CPU times: user 260 ms, sys: 0 ns, total: 260 ms
Wall time: 260 ms




##### Interesting material on string similarities

Approximate string matching:

https://medium.com/@wolfgarbe/fast-approximate-string-matching-with-large-edit-distances-in-big-data-2015-9174a0968c0b

Levenshtein distance using a trie:

http://stevehanov.ca/blog/?id=114

About jaccard distance:

https://python.gotrained.com/nltk-edit-distance-jaccard-distance/


Nice work on string similarities:

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.19.7158&rep=rep1&type=pdf

